In [1]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np

apex is not installed
apex is not installed
apex is not installed
apex is not installed


In [2]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기 (수정 1)
cfg = Config.fromfile('../../configs/_recycle_trash_base_/control/swin_b_cascade.py')

epoch = 'latest'

# cfg.optimizer['type'] = 'Adam'
# cfg.optimizer['lr'] = 0.0001
# del cfg.optimizer['momentum']

# dataset config 수정
cfg.data.test.classes = classes
cfg.data.test.test_mode = True

# 모델에 맞게 수정 (수정 2) - 아래는 faster_rcnn 기준
# cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize

# 결과가 저장된 곳 (수정 3)
cfg.work_dir = './check_point/64_swin_b_cascade_trainsetAll_3x'

#고정
cfg.seed=42
cfg.gpu_ids = [1]
# cfg.model.roi_head.bbox_head.num_classes = 10
cfg.optimizer_config.grad_clip = None
cfg.model.train_cfg = None

In [3]:
# build dataset & dataloader
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [4]:
# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'{epoch}.pth')

model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg')) # build detector
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu') # ckpt load

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

Use load_from_local loader


In [5]:
output = single_gpu_test(model, data_loader, show_score_thr=0.05) # output 계산

[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 4871/4871, 6.1 task/s, elapsed: 800s, ETA:     0s

In [6]:
# submission 양식에 맞게 output 후처리
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
img_ids = coco.getImgIds()

class_num = 10
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f'submission_{epoch}.csv'), index=None)
submission.head()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


,PredictionString,image_id
0,0 0.109484956 173.34131 424.64935 203.35062 44...,test/0000.jpg
1,0 0.8877369 641.194 649.2774 688.1987 841.0260...,test/0001.jpg
2,0 0.8935712 117.251595 294.85693 246.38419 595...,test/0002.jpg
3,0 0.10256065 0.0 0.0 1006.4777 764.7567 0 0.10...,test/0003.jpg
4,0 0.5635289 427.05157 410.7068 657.54193 573.1...,test/0004.jpg
